In [3]:
%gui qt5
%matplotlib qt5

import imp as reload_tool
import re
import os
import sh
from functools import partial
import pandas as pd

import matplotlib.pyplot   as plt
import matplotlib.gridspec as gridspec
import matplotlib.cm       as cm
import matplotlib          as mpl
from matplotlib import rcParams
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import NullFormatter

import numpy as np
import pycolleff.impedances as imp
import pycolleff.sirius as si
import pycolleff.colleff as colleff
import pycolleff.process_wakes as ems
import cppcolleff as cppcolef
import pyaccel
import pymodels as sirius

from IPython.display import Latex
from IPython.display import SVG

rcParams.update({
    'font.size': 22,'lines.linewidth':2,
    'axes.grid': True, 'grid.alpha': 0.5,
    'grid.linestyle': '--'})

# Study of Collective Effects for Sirius

First we load the appropriate model of the ring:

In [4]:
#Definition of default colormap function:
default_cm = lambda x: cm.brg(np.linspace(0,1,x+1 if isinstance(x,int) else len(x)+1))
# default_cm = lambda x: cm.nipy_spectral(np.linspace(0,1,x+1 if isinstance(x,int) else len(x)+1))

In [23]:
ring = si.create_ring(phase=2)
ring.nom_cur = 350e-3
print(ring)

Lattice Version             : SI.v20.01-s05.02.Phase2
Circumference [m]           :       518.396       
Revolution Period [us]      :        1.729        
Revolution Frequency [kHz]  :       578.3078      
Energy [GeV]                :         3.0         
Momentum Compaction         :       1.70e-04      
Harmonic Number             :         864         
Current [mA]                :        350.0        
Current per Bunch [mA]      :        0.405        
Synchrotron Tune            :      4.600e-03      
Tunes x/y                   :    49.110/14.170    
Damping Times x/y/e [ms]    :   10.6/ 12.5 /6.9   
Energy Spread               :       9.43e-04      
Bunch Length [mm]           :       2.66e+00      



# Longitudinal Plane

In [24]:
# Longitudinal modes
wr, RovQ, Q = np.array([
    [2466, 0.17, 16000], 
    [2532, 2.60, 1100],  
    [2606, 11.0, 240],
    [2695, 0.12, 22000],
    [2826, 6.57, 380],  
    [2979, 8.61, 270], 
    [3084, 1.93, 1200],
    [3180, 0.30, 7500],
    [3358, 0.86, 2400],
    [3594, 0.43, 4500],
    ]).T
wr *= 1e6 * 2 * np.pi
Rs = RovQ * Q

w = np.linspace(490, 30900, 100000) * 1e6*2*np.pi
w = np.hstack([-np.flipud(w), w])
Zl = imp.longitudinal_resonator(w=w, wr=wr, Rs=Rs, Q=Q)


# fig = plt.figure()
# plt.plot(w/2/np.pi*1e-9, Zl.real*1e-6)
# plt.plot(p/2/np.pi*1e-9, 0*p,'.')
# plt.plot(ph/2/np.pi*1e-9, 0*ph,'.')
# fig.show()

In [25]:
fig, ax1 = plt.subplots(figsize=(11,4))

ring = si.create_ring(phase=1)
# ring.nus = 2/578
curr = np.array([400, 350, 300])
colors = default_cm(len(curr))
# ax1.semilogy(wr/2/np.pi/1e9, Rs*1e-3, 'ko', label='SLS Rs')
Z = []
for cu, c in zip(curr, colors):
    ring.nom_cur = cu * 1e-3
    lab = f'{cu:0.1f} mA'
    Z.append(ring.longitudinal_cbi(w=w, Zl=Zl, m=1, inverse=True))
    freq = w/2/np.pi/1e9
    ax1.plot(freq, Z[-1]*1e-3, color=c, label=lab)

ax1.set_title('Min. Shunt Impedance (Rs) for CB Instability')
thres = 2*ring.nus() * ring.E/ring.mom_cmpct/ring.dampte/ 1e9
ax1.annotate(
    r'$I_0 = {0:.0f}\mathrm{{mA}} \longrightarrow \,\,$'.format(350) +
    r'$R_s [\mathrm{{k}}\Omega] < \frac{{{0:.1f}}}{{f[\mathrm{{GHz}}]}}$'.format(thres/0.35/1e3),
    xy=(0.3, 0.5), xycoords='axes fraction')
ax1.set_xlim([1, 10])
ax1.set_ylim([1, 70])
ax1.set_xlabel('Frequency [GHz]')
ax1.set_ylabel('Re$Z_L$ [k$\Omega$]')
ax1.legend(loc='upper right', fontsize='x-small')
fig.tight_layout()
plt.show()

# fig.savefig('long_cbi_minimum_impedance.png')

## Robinson Instability

In [28]:
# Longitudinal modes
def get_impedance_robinson(ws, dtune=45.897e3, RovQ=90, Q=4e8):
    Rs = RovQ * Q
    wr = frf + dtune
    wr *= 2*np.pi

    Zl = imp.longitudinal_resonator(w=ws, wr=wr, Rs=Rs, Q=Q)
    return Zl

frf = 3*ring.f0*ring.harm_num
w = np.linspace(1400, 1800, 100000)
w = np.r_[w, 3000] * 1e6*2*np.pi
w = np.r_[-np.flipud(w), w]

# fig = plt.figure()
# plt.plot(w/2/np.pi*1e-9, Zl.real*1e-6)
# plt.plot(p/2/np.pi*1e-9, 0*p,'.')
# plt.plot(ph/2/np.pi*1e-9, 0*ph,'.')
# fig.show()

In [36]:
RovQ = 90
dtunes = np.linspace(36, 48, 50) * 1e3

# RovQ = 180
# dtunes = np.linspace(75, 95, 50) * 1e3

grates, tshifts = [], []
for dtune in dtunes:
    func = partial(get_impedance_robinson, dtune=dtune, RovQ=RovQ)
    deltaw, wp, Zl_interp, Zleff = ring.longitudinal_cbi(w=w, Zl=func, m=1, full=True)
    deltaw *= ring.f0 * ring.nus() * 2* np.pi
    grates.append(deltaw.imag.max())
    tshifts.append(deltaw.real.max()/2/np.pi)

grates = np.array(grates)
tshifts = np.array(tshifts)

In [16]:
plt.figure()
Zl = func(w)
plt.semilogy(w/2/np.pi, Zl.real)
plt.semilogy(wp/2/np.pi, Zl_interp.real, 'o')
plt.axvline(frf, color='k', ls='--')
plt.show()

In [17]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
ax2.set_xlabel('Coupled Bunch Mode')
ax1.set_ylabel('Growth Rate [1/s]')
ax2.set_ylabel('Frequency Shift [Hz]')

ax1.plot(deltaw.imag, 'o')
ax2.plot(deltaw.real/2/np.pi, 'o')
fig.tight_layout()
plt.show()

In [37]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
ax2.set_xlabel('Cavity Detune [kHz]')
ax1.set_ylabel('Growth Rate [1/s]')
ax2.set_ylabel('Frequency Shift [Hz]')

ax1.plot(dtunes/1e3, grates)
ax2.plot(dtunes/1e3, tshifts)
fig.tight_layout()
plt.show()

# Transverse Plane

In [26]:
# Transverse modes
wr, RovQ, Q = np.array([
    [1721, 20.0 , 6500],
    [1723, 21.8 , 600],
    [1935, 0.01 , 1.1e7],
    [2056, 255  , 510],
    [2103, 27.6 , 4710],
    [2148, 437  , 300],
    [2303, 10.1 , 12900],
    [2503, 11.1 , 11700],
    [2712, 63.8 , 2040],
    [2865, 10.3 , 12600],
    ]).T
wr *= 1e6 * 2 * np.pi
Rs = RovQ * Q

p = np.arange(0,900)*ring.w0
ph = np.arange(0,2)*ring.w0*ring.nbun
w = np.linspace(490, 30900, 100000) * 1e6*2*np.pi
w = np.hstack([-np.flipud(w), w])
Zt = imp.transverse_resonator(w=w, wr=wr, Rs=Rs, Q=Q)
# plt.figure()
# plt.plot(w/2/np.pi*1e-9, Zt.real*1e-6)
# plt.plot(p/2/np.pi*1e-9, 0*p,'.')
# plt.plot(ph/2/np.pi*1e-9, 0*ph,'.')
# plt.show()

In [27]:
fig, ax1 = plt.subplots(figsize=(9,5))
gs = gridspec.GridSpec(1, 1)

# ax1.semilogy(wr/2/np.pi/1e9, Rs*1e-6, 'ko', label='SLS Rs')

ring = si.create_ring(phase=1)
ring.chromx = 0
ring.chromy = 0
# ring.nus = 2/578
curr = np.array([400, 350, 300])

colors = default_cm(len(curr))
betat = 4.0
for cu, c in zip(curr, colors):
    ring.nom_cur = cu*1e-3
    lab = f'{cu:0.1f} mA'
    Z = ring.transverse_cbi(w=w, Zt=Zt, m=0, inverse=True, plane='y')/betat
    ax1.plot(w/2/np.pi/1e9, Z*1e-3, color=c, label=lab)
    
ax1.set_title('Min. Shunt Impedance (Rs) for CB Instability')
thres = 2*ring.E/ring.dampty/ ring.f0 / betat
ax1.annotate(
    r'$I_0 = {0:.0f}\mathrm{{mA}}, \beta_y = {1:.1f} \mathrm{{m}} \longrightarrow \,\,$'.format(350, betat) +
    r'$R_y < {0:.1f} \mathrm{{k}}\Omega/m$'.format(thres/0.35/1e3),
    xy=(0.2, 0.8), xycoords='axes fraction', fontsize='small')

ax1.set_xlim([1, 10])
ax1.set_ylim([400, 800])
ax1.set_xlabel('Frequency [GHz]')
ax1.set_ylabel('Re$Z_y$ [k$\Omega$]')
ax1.legend(loc='upper left', fontsize='x-small')
fig.tight_layout()
plt.show()